In [1]:
cd ..

/home/alex/gargaml


In [2]:
ls

build/        docs/      gargaml/           pyproject.toml    setup.py
CHANGELOG.md  env/       gargaml.egg-info/  README.md         tests/
dist/         examples/  LICENSE            requirements.txt  utils/


In [3]:
from gargaml import * 

In [4]:
df = Data.Load.food()

In [5]:
df.dtypes

url                             object
created_datetime                object
product_name                    object
quantity                        object
brands                          object
categories                      object
categories_fr                   object
countries_tags                  object
ingredients_text                object
serving_size                    object
additives_n                    float64
additives_tags                  object
ingredients_from_palm_oil_n    float64
nutrition_grade_fr              object
pnns_groups_1                   object
pnns_groups_2                   object
main_category                   object
energy_100g                    float64
fat_100g                       float64
saturated-fat_100g             float64
carbohydrates_100g             float64
sugars_100g                    float64
fiber_100g                     float64
proteins_100g                  float64
salt_100g                      float64
sodium_100g              

In [7]:
df.created_datetime  = pd.to_datetime(df.created_datetime)

In [9]:
df.dtypes

url                                         object
created_datetime               datetime64[ns, UTC]
product_name                                object
quantity                                    object
brands                                      object
categories                                  object
categories_fr                               object
countries_tags                              object
ingredients_text                            object
serving_size                                object
additives_n                                float64
additives_tags                              object
ingredients_from_palm_oil_n                float64
nutrition_grade_fr                          object
pnns_groups_1                               object
pnns_groups_2                               object
main_category                               object
energy_100g                                float64
fat_100g                                   float64
saturated-fat_100g             

In [13]:
cols = ["created_datetime", "nutrition_grade_fr", "pnns_groups_1", "sugars_100g"]
df = df.loc[:, cols]
df = df.loc[df.nutrition_grade_fr.notna(), :]
df = df.loc[df.pnns_groups_1.notna(), :]
df = df.loc[df.sugars_100g.notna(), :]
df

,created_datetime,nutrition_grade_fr,pnns_groups_1,sugars_100g
3,2015-01-25 21:35:01+00:00,e,unknown,57.50
17,2012-08-15 09:48:28+00:00,e,Beverages,10.40
21,2015-06-06 10:28:06+00:00,e,Beverages,10.40
22,2016-10-20 07:19:01+00:00,d,Sugary snacks,26.00
24,2016-12-15 22:15:44+00:00,d,Sugary snacks,28.00
...,...,...,...,...
98597,2015-03-14 14:32:26+00:00,c,Sugary snacks,0.12
98607,2016-12-22 07:02:13+00:00,b,Sugary snacks,0.10
98622,2017-01-23 16:57:57+00:00,c,Salty snacks,1.80
98632,2015-07-25 09:47:06+00:00,a,Cereals and potatoes,2.60


In [34]:
gb = df.groupby([df.created_datetime.dt.year, df.created_datetime.dt.month]).agg({"sugars_100g" : "count"})

# gb.reset_index()
# gb.reset_index()
gb.index.name = ["year", "month"]
gb

TypeError: MultiIndex.name must be a hashable type

In [55]:
gb = df.groupby(df['created_datetime'].dt.to_period('M')).agg({"sugars_100g" : ["mean", "sum", "count"]})
gb = gb.reset_index()

gb.columns = ["created_datetime", "p_mean", "ca", "nb"]
gb.created_datetime = gb.created_datetime.astype(str)
gb

/tmp/ipykernel_259958/2051676159.py:1: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,created_datetime,p_mean,ca,nb
0,2012-01,49.300000,49.300,1
1,2012-02,17.450000,139.600,8
2,2012-03,13.714286,1152.000,84
3,2012-04,13.406404,1193.170,89
4,2012-05,12.818395,3114.870,243
...,...,...,...,...
59,2016-12,11.465124,10605.240,925
60,2017-01,13.724072,13175.109,960
61,2017-02,12.620137,18564.221,1471
62,2017-03,12.734955,14110.330,1108


In [53]:
px.line(gb, x="created_datetime", y="nb")

In [ ]:
df.energy_100g.describe()

In [ ]:
sns.boxplot(df.energy_100g)

In [ ]:
df = df.loc[df.energy_100g< 10_000]
df

In [ ]:
df

In [ ]:
df.energy_100g.describe()

In [ ]:
df.dtypes

In [ ]:


df.loc[df.product_name.fillna("").str.lower().str.contains("huile d'")]

In [ ]:
df = df.loc[df.energy_100g<4_000]

In [ ]:
df.sort_values("energy_100g", ascending=False).round(2)

In [ ]:
df.url

In [ ]:
df.pnns_groups_1

In [ ]:
df.pnns_groups_1.value_counts(normalize=True, dropna=False).round(1)

In [ ]:
replace_dict  = {"unknown" : "INCONNU", "None" : "INCONNU", None : "INCONNU", np.nan : "INCONNU"}
df.pnns_groups_1 = df.pnns_groups_1.replace(replace_dict)
df.pnns_groups_1.value_counts(normalize=True, dropna=False).round(1)

In [ ]:
df.pnns_groups_1 = df.pnns_groups_1.str.lower()

In [ ]:
df.pnns_groups_1.value_counts(normalize=True, dropna=False).round(1).sort_index()

In [ ]:
df.pnns_groups_1 = df.pnns_groups_1.str.replace("-", " ").str.strip()
df.pnns_groups_1.value_counts(normalize=True, dropna=False).round(1).sort_index()

In [ ]:
tmp = df.isna().mean().sort_values(ascending=False).round(2)
tmp = tmp[tmp<0.3]
tmp.index

In [ ]:
df = df.loc[:, tmp.index]
df

In [ ]:
tmp = df.isna().mean(axis=1).sort_values(ascending=False).round(2)
tmp = tmp[tmp<0.3]
tmp.index

In [ ]:
df = df.loc[tmp.index, : ]
df

In [ ]:
dd = {"mois" : [1,2,3, 1,2,3, 1,2,3 ]"pdt" : [0,1,2,0,1,2,0,1,2, ] , "ca" [10,10, 10, 20, 20, 20, 30, 30,0]}

In [ ]:
cols = ['fiber_100g', 'carbohydrates_100g', 'fat_100g', 
       'nutrition_grade_fr', 'nutrition-score-fr_100g', 'saturated-fat_100g',
       'sugars_100g','salt_100g', 'product_name',
       'proteins_100g', 'pnns_groups_1', 'energy_100g',
       'countries_tags']

df = df.loc[:, cols]
df

In [ ]:
df = df.loc[df.countries_tags.fillna("").str.contains("fr"), :]
df

In [ ]:
# HYPER FILTRE

In [ ]:
df = df.loc[df.nutrition_grade_fr.notna(), : ]

In [ ]:
df

In [ ]:
df

In [ ]:
df_num = df.select_dtypes(include=np.number)
df_cat = df.select_dtypes(exclude=np.number)

In [ ]:
df_num

In [ ]:
df_cat

In [ ]:
# OPTION 1
imputer = KNNImputer()
_df_num = imputer.fit_transform(df_num)
_df_num


In [ ]:
_df_num = pd.DataFrame(_df_num, columns=df_num.columns)
_df_num

In [ ]:
_df_num.isna().sum()

In [ ]:
pnns_list = df.pnns_groups_1.unique()
pnns_list


In [ ]:
value_list = []

# iter on pnns
for pnns in pnns_list : 
    # select sub df for this pnns
    _df = df.loc[df.pnns_groups_1 == pnns ]

    # for each num col
    _df = _df.select_dtypes(include=np.number)
    for col in _df.columns : 
        # compute mean
        _mean = _df[col].mean().round(2)
        # add to value list
        tu = (pnns, col, _mean)
        value_list.append(tu)

In [ ]:
value_list

In [ ]:
# OPTION 2
for tu in value_list : 
    pnns, col, val = tu 
    idx = df[col].isna() & (df.pnns_groups_1== pnns)
    df.loc[idx, :] = val

In [ ]:
_df = df.select_dtypes(include=np.number)
_df

In [ ]:
_df

In [ ]:
_df.mean().round(2).to_dict()

In [ ]:
for col, val in _df.mean().to_dict().items() : 
    print(col)
    print(val)

    df[col].fillna(val)

In [ ]:
# OPtion 3


cols = ["pnns_groups_1",] + list(df.select_dtypes(np.number).columns)
cols

In [ ]:
_df = df.loc[:, cols]
_df

In [ ]:
_df_dummies = pd.get_dummies(_df)
cols_dummies = _df_dummies.columns
_df_dummies

In [ ]:
_df_dummies = KNNImputer().fit_transform(_df_dummies)
_df_dummies = pd.DataFrame(_df_dummies, columns=cols_dummies)

In [ ]:
_df_dummies